# Generation of Graphene sheets
## This code is use to generate extended or trimed graphene sheet from the original one

In [1]:
import os
import numpy as np

In [2]:
def extended_graphene (file_name,out_file,nx, ny, dz, ncat, rho_n, periodic):
    #the original graphene sheet has 2.5 carbon rings through x axis, 3 carbon rings through y axis
    #nx = number of times extending through x axis 
    #ny = number of times extending through y axis 
    #dz = distance between two sheets through z-axis
    #rho_n = water number density in 1000 cubic Angstroms 
    #to have original sheet nx=ny=0
    
    file = os.path.join(f"{file_name}.xyz")

    atom= []
    x=[]
    y=[]
    z= []
    with open(file, 'r') as file:
        xyz= file.readlines()[2:]
        for indx,line in enumerate(xyz):
            atom.append(str(line.split()[0]))
            x.append(float(line.split()[1]))
            y.append(float(line.split()[2]))
            z.append(float(line.split()[3]))

    #rx=2.52584 #width of the C ring through x-axis
    rx=2.533 #mean width of the C ring through x-axis
    ry=1.488 #C-C bond distance of graphene through y-axis

    #for extending sheet
    dx= 3*rx  #the extended sheet should start from +dx distance through x-axis compared to original.xyz
    dy= 9*ry  #the extended sheet should start from +dy distance through y-axis compared to original.xyz
    dz= dz  #distance between two sheets through z-axis

    ddx=dx*(nx+1) #length of the extended graphene sheet in X-direction
    ddy=dy*(ny+1) #length of the extended graphene sheet in Y-direction

    #box content
    #vC4= 101.33
    vC6_nmeth= 192.87
    vC4_nmeth=158.42 #cubic angstroms
    vC2_nmeth=121.71
    vCat=vC2_nmeth
    vWater= 19.40
    vH=7.24
    nH=18
    vOH= 17.14
    #vCl= 23.70
    vbox =ddx*ddy*dz
    v_rest= vbox -ncat*(vCat+ vOH + vH*((nx+1)*(ny+1)*(2*nH-1)))
    nwater=v_rest/vWater
    
    atom_new= []
    x_new=[]
    y_new=[]
    z_new= []

    #number of atoms in original graphene sheet
    natom= len(atom)
    with open(os.path.join(f"{out_file}.xyz"), 'w') as new_file:
        #for IQmol
        new_file.write(str(natom*(nx+1)*(ny+1)*2)+"\n"+"\n")
        #sheet 1
        #original sheet with extended sheet

        for a in range(nx+1):  
            for b in range(ny+1):  
                for i in range(natom):
                    new_file.write(atom[i] + " " + str(round((x[i]+(a*dx)),6)) + " " + str(round(y[i]+(b*dy),6)) + " " + str(z[i])+"\n")
                    atom_new.append(atom[i])
                    x_new.append(round((x[i]+(a*dx)),6))
                    y_new.append(round(y[i]+(b*dy),6))
                    z_new.append(z[i])
                    
        #number of atoms in extended graphene sheet
        exatom= len(atom_new)            
        #sheet 2
        for i in range(exatom):
            new_file.write(atom_new[i] + " " + str(x_new[i]) + " " + str(y_new[i]) + " " + str(round(z_new[i]+(dz),6))+"\n")
            atom_new.append(atom_new[i])
            x_new.append(x_new[i])
            y_new.append(y_new[i])
            z_new.append(round(z_new[i]+(dz),6))

    #check with periodic boundary conditions
    nnatom= len(atom_new) #number of atoms in extended graphene sheets


    with open(os.path.join(f"{periodic}.xyz"), 'w') as new_file:
        #for IQmol
        new_file.write(str(nnatom*5)+"\n"+"\n")
        #extended two graphene sheets
        for i in range(nnatom):
            new_file.write(atom_new[i] + " " + str(x_new[i]) + " " + str(y_new[i]) + " " + str(z_new[i])+"\n")
    
        #periodic box(+ddx)
        for i in range(nnatom):
            new_file.write(atom_new[i] + " " + str(round((x_new[i]+ddx),6)) + " " + str(y_new[i]) + " " + str(z_new[i])+"\n")
        #periodic box(-ddx)
        for i in range(nnatom):
            new_file.write(atom_new[i] + " " + str(round((x_new[i]-ddx),6)) + " " + str(y_new[i]) + " " + str(z_new[i])+"\n")
        #periodic box(+ddy)
        for i in range(nnatom):
            new_file.write(atom_new[i] + " " + str(x_new[i]) + " " + str(round((y_new[i]+ddy),6)) + " " + str(z_new[i])+"\n")
        #periodic box(-ddy)
        for i in range(nnatom):
            new_file.write(atom_new[i] + " " + str(x_new[i]) + " " + str(round((y_new[i]-ddy),6)) + " " + str(z_new[i])+"\n")

    print('Number of the atoms in two graphene sheets:', (natom*(nx+1)*(ny+1)*2))
    print('Length of the box through X axis:', ddx, 'Angstroms')
    print('Length of the box through Y axis:', ddy, 'Angstroms')
    print('Length of the box through Z axis:', dz, 'Angstroms')
    print('Volume of the box:', vbox, 'Cubic Angstroms')
    print('Volume of the box without ions and Hs:', v_rest, 'Cubic Angstroms')
    print('Maximum number of water molecules fit in the box:', int(nwater))

    #bulk water number density could be fall in between 7.9-33.4 water molecules in 1000 cubic Angstroms
    
    print(f'Number of water molecules should have in the box to have {rho_n}/1000 cubic Angstroms:', int(v_rest/1000*rho_n))
    #print('Minimum number density of water in 1000 Cubic Angstroms:', float(int(v_rest/1000*rho_n))/v_rest*1000)
    
    #print('Maximum number of water molecules in the box according to the bulk water:', int(v_rest/1000*31.7))
    #print('Maximum number density of water in 1000 Cubic Angstroms:', float(int(v_rest/1000*31.7))/v_rest*1000)


In [3]:
extended_graphene('original', 'graphene_box3', 3,1, 15, 2, 10, 'periodic_box3')

Number of the atoms in two graphene sheets: 1152
Length of the box through X axis: 30.396 Angstroms
Length of the box through Y axis: 26.784 Angstroms
Length of the box through Z axis: 15 Angstroms
Volume of the box: 12211.896959999998 Cubic Angstroms
Volume of the box without ions and Hs: 7879.796959999998 Cubic Angstroms
Maximum number of water molecules fit in the box: 406
Number of water molecules should have in the box to have 10/1000 cubic Angstroms: 78


In [4]:
extended_graphene('original', 'graphene_box2', 2,0, 15, 2, 10, 'periodic_box2')

Number of the atoms in two graphene sheets: 432
Length of the box through X axis: 22.797 Angstroms
Length of the box through Y axis: 13.392 Angstroms
Length of the box through Z axis: 15 Angstroms
Volume of the box: 4579.461359999999 Cubic Angstroms
Volume of the box without ions and Hs: 2781.361359999999 Cubic Angstroms
Maximum number of water molecules fit in the box: 143
Number of water molecules should have in the box to have 10/1000 cubic Angstroms: 27


In [10]:
extended_graphene('original', 'graphene_box1', 1,0, 15, 1, 10,'periodic_box_1')

Number of the atoms in two graphene sheets: 288
Length of the box through X axis: 15.198 Angstroms
Length of the box through Y axis: 13.392 Angstroms
Length of the box through Z axis: 15 Angstroms
Volume of the box: 3052.9742399999996 Cubic Angstroms
Volume of the box without ions and Hs: 2407.3242399999995 Cubic Angstroms
Maximum number of water molecules fit in the box: 124
Number of water molecules should have in the box to have 10/1000 cubic Angstroms: 24
